# Minor Project Coding using Python

# Importing modules

In [1]:
import pygame
import cv2
import mediapipe as mp
import numpy as np

pygame 2.5.2 (SDL 2.28.3, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html



# Initialize Pygame

In [2]:
pygame.init()
clock = pygame.time.Clock()
screen_width, screen_height = 800, 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Air Touch Game")

white = (255, 255, 255)
black = (0, 0, 0)
red = (255, 0, 0)

# Image loading

In [3]:
car_image = pygame.image.load("earth1.jpeg")
car_image = pygame.transform.scale(car_image, (70, 140))  
car_x, car_y = screen_width // 2, screen_height - 150
car_speed = 5

background_image = pygame.image.load("back2.jpeg")
background_image = pygame.transform.scale(background_image, (screen_width, screen_height))

stone_image = pygame.image.load("missile2.png")
stone_image = pygame.transform.scale(stone_image, (50, 50))

# Loading BGM

In [4]:
pygame.mixer.music.load("avengers1.mp3")  
pygame.mixer.music.play()  


def is_music_playing():
    return pygame.mixer.music.get_busy()

# Initialize Mediapipe

In [5]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# OpenCV setup

In [6]:
cap = cv2.VideoCapture(0)

stone_count = 5
stones = [{"x": np.random.randint(0, screen_width - 50), "y": np.random.randint(-600, -50)} for _ in range(stone_count)]
stone_speed = 5

score = 0
high_score = 0
font = pygame.font.Font(None, 36)

def draw_car(x, y, image):
    screen.blit(image, (x, y))

def draw_stone(x, y):
    screen.blit(stone_image, (x, y))

def show_text(text, x, y, color):
    screen_text = font.render(text, True, color)
    screen.blit(screen_text, (x, y))

def main():
    global car_x, stones, score, high_score
    running = True
    music_played_once = False

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        if not is_music_playing() and not music_played_once:
            music_played_once = True
            pygame.time.wait(500)  
            pygame.mixer.music.load("poco1.mp3")  
            pygame.mixer.music.play(-1)  
       
        success, frame = cap.read()
        if not success:
            continue

        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0].landmark

            index_finger_tip_x = int(hand_landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * screen_width)
            if index_finger_tip_x < screen_width // 3:
                car_x -= car_speed
            elif index_finger_tip_x > screen_width * 2 // 3:
                car_x += car_speed

            if car_x < 0:
                car_x = 0
            elif car_x > screen_width - car_image.get_width():
                car_x = screen_width - car_image.get_width()

        for stone in stones:
            stone["y"] += stone_speed
            if stone["y"] > screen_height:
                stone["y"] = np.random.randint(-600, -50)
                stone["x"] = np.random.randint(0, screen_width - 50)
                score += 1  

        for stone in stones:
            if car_y < stone["y"] + 50 and car_y + 140 > stone["y"] and car_x < stone["x"] + 50 and car_x + 70 > stone["x"]:
                running = False  
        screen.blit(background_image, (0, 0))
        draw_car(car_x, car_y, car_image)
        for stone in stones:
            draw_stone(stone["x"], stone["y"])
        show_text(f"Score: {score}", 10, 10, black)
        show_text(f"High Score: {high_score}", 10, 50, black)
        pygame.display.update()
        clock.tick(30)

        
        cv2.imshow("Hand Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    if score > high_score:
        high_score = score
        show_text("You got a High Score!", screen_width // 2 - 100, screen_height // 2, red)
    else:
        show_text("You Lose!", screen_width // 2 - 50, screen_height // 2, red)
    pygame.display.update()
    pygame.time.wait(3000)

    pygame.quit()
    cap.release()
    cv2.destroyAllWindows()

main()
